# Advanced CharRNN Techniques!

- Some more techniques for making CharRNNs!
- Training and generating directly from integer sequences (no one-hot encoding!)
- Making a one-in, one-out model!

Starting out with the same data and code.

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"
## Much borrowed from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
import keras
from keras import Sequential, Input
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import Embedding
from keras.optimizers import RMSprop
from keras.utils import get_file
from keras.models import load_model
import numpy as np
import random
import sys
from urllib.request import urlopen

# Local Jupyter Only
# text = open("../datasets/startrekepisodes.txt").read().lower()
# print('corpus length:', len(text))

# Colab
ST_EPISODES_URL = "https://raw.githubusercontent.com/cpmpercussion/creative-prediction/master/datasets/startrekepisodes.txt"
text = urlopen(ST_EPISODES_URL).read().decode("utf-8").lower()

chars = sorted(list(set(text)))
vocabulary_size = len(chars)
print('total chars:', vocabulary_size)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Setup Training Data

- Cut up the corpus into sequences of 40 characters.
- Translate characters to integer indices

In [ ]:
# cut the text in semi-redundant sequences of maxlen characters
seq_len = 30
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - seq_len, step):
    sentences.append(text[i: i + seq_len])
    next_chars.append(text[i + seq_len])
print('nb sequences:', len(sentences))

X = np.zeros((len(sentences), seq_len), dtype=int)
y = np.zeros(len(sentences), dtype=int)

for i in range(len(sentences)):
    X[i] = np.array([char_indices[x] for x in sentences[i]])
    y[i] = char_indices[next_chars[i]]

print("Done preparing training corpus, shapes of sets are:")
print("X shape: " + str(X.shape))
print("y shape: " + str(y.shape))
print("Vocabulary of characters:", vocabulary_size)

print()
print("X[0]:", X[0])
print("y[0]:", y[0])

## Model

- Model has one hidden layer of 128 LSTM cells.
- Input layer is an Embedding to convert from indices to a vector encoding automatically (common trick - but does it work?)

In [ ]:
layer_size = 128
dropout_rate = 0.5
# build the model: a single LSTM
print('Build model...')
model_train = Sequential()

# The embedding layer maps from a given number of indexes to a vectors of a certain length.
# In our case, from vocabulary_size to layer_size.
model_train.add(Input(shape=(len(chars),)))
model_train.add(Embedding(vocabulary_size, layer_size))

# LSTM part  - this time 2 layer LSTM
model_train.add(LSTM(layer_size, return_sequences=True))
model_train.add(LSTM(layer_size))

# Project back to vocabulary
model_train.add(Dense(vocabulary_size))
model_train.add(Activation('softmax'))
model_train.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')
model_train.summary()

## Training

- Proceeds as before!
- No frills this time.

In [ ]:
# Training the Model.
model_train.fit(X, y, batch_size=64, epochs=50)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(model_train.history.history['loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
# Save model if necessary
model_train.save("keras-startrek-LSTM-model.keras")

## Test the Model

- Take a quote then add 400 characters.

### Make a Decoder model

- Needs input length of 1.
- Needs batch size of 1
- Needs LSTM to be stateful
- check that params is the same as model_train

In [ ]:
# Load model if necessary.
model_train = keras.saving.load_model("keras-startrek-LSTM-model.keras")

In [ ]:
# Build a decoding model (input length 1, batch size 1, stateful)

model_dec = Sequential()
model_dec.add(Input(shape=(len(chars),), batch_size=1))
model_dec.add(Embedding(vocabulary_size, layer_size))

# LSTM part
model_dec.add(LSTM(layer_size, stateful=True, return_sequences=True))
model_dec.add(LSTM(layer_size, stateful=True))

# project back to vocabulary
model_dec.add(Dense(vocabulary_size, activation='softmax'))
model_dec.summary()

# set weights from training model
model_dec.set_weights(model_train.get_weights())

In [ ]:
## Sampling function

def sample_model(seed, model_name, length=400):
    '''Samples a charRNN given a seed sequence.'''
    generated = ''
    sentence = seed.lower()[:]
    generated += sentence
    print("Seed: ", generated)
    
    for i in range(length):
        x = np.array([char_indices[n] for n in sentence])
        x = np.reshape(x,(1,1))
        preds = model_name(x)[0] # for jax
        # preds = model_name(x).numpy()[0] # for tensorflow

        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]
        
        generated += next_char
        print(next_char, end='')
        sentence = sentence[1:] + next_char
    return generated

In [ ]:
# Sample 1000 characters from the model using a random seed from the vocabulary.
g = sample_model(indices_char[random.randint(0,vocabulary_size-1)], model_dec, 1000)